In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import sys
import torch
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import neural_network_lyapunov
import neural_network_lyapunov.relu_system as relu_system
import neural_network_lyapunov.lyapunov as lyapunov
import neural_network_lyapunov.encoders as encoders
import neural_network_lyapunov.pybullet_data_generation as pybullet_data_generation
import neural_network_lyapunov.dynamics_learning as dynamics_learning

## Loading options

In [ ]:
folder = "pendulum_1013"

variant = "unstable"
device = 'cuda:0'

# variant = "stable"
# device = 'cuda:1'

sys.path.insert(0, os.path.join(os.path.dirname(neural_network_lyapunov.__file__), "..", "notebooks", folder))
import options
opt = dynamics_learning.DynamicsLearningOptions(options.default)
opt.set_options(options.variants[variant])

In [ ]:
print(opt.lyapunov_loss_weight)
print(opt.lyapunov_loss_freq)
print(opt.lyapunov_loss_optimal)
print(opt.encoder_class)
print(opt.validation_rollouts_N)
print(opt.V_lambda)
print(opt.z_lo)
print(opt.dyn_nn_depth)
print(opt.dyn_nn_width)
print(opt.z_equilibrium)
print(opt.validation_rollouts_N)

## Generating data

In [ ]:
pbsg = pybullet_data_generation.PybulletSampleGenerator(opt.world_cb, opt.joint_space, image_width=opt.image_width, image_height=opt.image_height,
                                                        grayscale=opt.grayscale, dtype=opt.dtype,
                                                        camera_eye_position=opt.camera_eye_position,
                                                        camera_target_position=opt.camera_target_position,
                                                        camera_up_vector=opt.camera_up_vector)

In [ ]:
x_data, x_next_data, X_data, X_next_data = pbsg.generate_dataset(opt.pybullet_x_lo, opt.pybullet_x_up, opt.dataset_dt, opt.dataset_N, opt.num_samples)

In [ ]:
x_data_ = dynamics_learning.add_noise(x_data, opt.pybullet_noise)
x_next_data_ = dynamics_learning.add_noise(x_next_data, opt.pybullet_noise)
x_train_dataloader, x_validation_dataloader = dynamics_learning.get_dataloaders(x_data_, x_next_data_, opt.batch_size, opt.validation_ratio)
X_train_dataloader, X_validation_dataloader = dynamics_learning.get_dataloaders(X_data, X_next_data, opt.batch_size, opt.validation_ratio)

In [ ]:
x_data_rollouts_, _, _, _ = pbsg.generate_dataset(opt.pybullet_x_lo, opt.pybullet_x_up, opt.dataset_dt, 1, opt.validation_max_rollouts)
X_rollouts, x_rollouts = dynamics_learning.data_to_rollouts(pbsg, x_data_rollouts_, opt.dataset_dt, opt.validation_rollouts_N)

In [ ]:
torch.save(x_data, os.path.join(folder, "x_data"))
torch.save(x_next_data, os.path.join(folder, "x_next_data"))
torch.save(X_data, os.path.join(folder, "X_data"))
torch.save(X_next_data, os.path.join(folder, "X_next_data"))
torch.save(X_rollouts, os.path.join(folder, "X_rollouts"))
torch.save(x_rollouts, os.path.join(folder, "x_rollouts"))

In [ ]:
torch.save(x_train_dataloader, os.path.join(folder, "x_train_dataloader"))
torch.save(x_validation_dataloader, os.path.join(folder, "x_validation_dataloader"))
torch.save(X_train_dataloader, os.path.join(folder, "X_train_dataloader"))
torch.save(X_validation_dataloader, os.path.join(folder, "X_validation_dataloader"))

In [ ]:
x_data = torch.load(os.path.join(folder, "x_data"))
x_next_data = torch.load(os.path.join(folder, "x_next_data"))
X_data = torch.load(os.path.join(folder, "X_data"))
X_next_data = torch.load(os.path.join(folder, "X_next_data"))
X_rollouts = torch.load(os.path.join(folder, "X_rollouts"))
x_rollouts = torch.load(os.path.join(folder, "x_rollouts"))

In [ ]:
x_train_dataloader = torch.load(os.path.join(folder, "x_train_dataloader"))
x_validation_dataloader = torch.load(os.path.join(folder, "x_validation_dataloader"))
X_train_dataloader = torch.load(os.path.join(folder, "X_train_dataloader"))
X_validation_dataloader = torch.load(os.path.join(folder, "X_validation_dataloader"))

In [ ]:
# plot some random sample
i = np.random.choice(X_data.shape[0], 1)[0]
pybullet_data_generation.show_sample(X_data[i,:], X_next_data[i,:])

In [ ]:
# plot some random rollout
i = np.random.choice(X_data.shape[0], 1)[0]
# X_traj, x_traj = pbsg.generate_rollout(x_data[i,:], opt.dataset_dt, 2)
X_traj, x_traj = pbsg.generate_rollout(torch.tensor(opt.z_equilibrium, dtype=opt.dtype), opt.dataset_dt, 2)
for n in range(X_traj.shape[0]):
    pybullet_data_generation.show_sample(X_traj[n, :])

In [ ]:
len(x_rollouts)

## Learning in state space

In [ ]:
dyn_nn_model = dynamics_learning.get_ff_network(opt.dtype, opt.z_dim, opt.z_dim, opt.dyn_nn_width, opt.dyn_nn_depth)
lyap_nn_model = dynamics_learning.get_ff_network(opt.dtype, opt.z_dim, 1, opt.lyap_nn_width, opt.lyap_nn_depth)

relu_sys = relu_system.AutonomousReLUSystemGivenEquilibrium(opt.dtype, opt.z_lo, opt.z_up,
                                                            dyn_nn_model, opt.z_equilibrium)
lyap = lyapunov.LyapunovDiscreteTimeHybridSystem(relu_sys, lyap_nn_model)

dyn_learner = dynamics_learning.StateSpaceDynamicsLearning(x_train_dataloader, x_validation_dataloader, relu_sys, lyap, opt)

In [ ]:
dyn_learner.train(50, validate=True, device=device, save_rate=5, save_path=os.path.join(folder, variant))

In [ ]:
i = np.random.choice(x_data.shape[0], 1)[0]
x0 = x_data[i, :]
x_traj, V_traj = dyn_learner.rollout(x0, opt.validation_rollouts_N)
plt.plot(x_traj)
_, x_traj_exp = pbsg.generate_rollout(x0, opt.dataset_dt, opt.validation_rollouts_N)
plt.plot(x_traj_exp)
plt.legend(['predicted', 'predicted', 'expected', 'expected'])
plt.show()
plt.plot(V_traj)
plt.show()

In [ ]:
# validation over rollouts
dyn_learner.rollout_validation(x_rollouts)

In [ ]:
torch.save(dyn_learner.relu_system.dynamics_relu, os.path.join(folder, variant, "dynamics"))
torch.save(dyn_learner.lyapunov.lyapunov_relu, os.path.join(folder, variant, "lyapunov"))

## Learning in image space

In [ ]:
dyn_nn_model = dynamics_learning.get_ff_network(opt.dtype, opt.z_dim, opt.z_dim, opt.dyn_nn_width, opt.dyn_nn_depth)
lyap_nn_model = dynamics_learning.get_ff_network(opt.dtype, opt.z_dim, 1, opt.lyap_nn_width, opt.lyap_nn_depth)

relu_sys = relu_system.AutonomousReLUSystemGivenEquilibrium(opt.dtype, opt.z_lo, opt.z_up,
                                                            dyn_nn_model, opt.z_equilibrium)
lyap = lyapunov.LyapunovDiscreteTimeHybridSystem(relu_sys, lyap_nn_model)
encoder = opt.encoder_class(opt.z_dim, opt.image_width, opt.image_height, opt.grayscale)
decoder = opt.decoder_class(opt.z_dim, opt.image_width, opt.image_height, opt.grayscale)

dyn_learner = dynamics_learning.LatentSpaceDynamicsLearning(X_train_dataloader, X_validation_dataloader,
                                                            relu_sys, lyap, opt,
                                                            encoder, decoder,
                                                            use_bce=opt.use_bce, use_variational=opt.use_variational)

In [ ]:
# pre-train encoder-decoder
import time
start = time.time()
dyn_learner.train_encoder(100, validate=True, device=device)
end = time.time()
print(end - start)

In [ ]:
# plot some encoding-decoding
i = np.random.choice(X_data.shape[0], 1)[0]
pybullet_data_generation.show_sample(X_data[i,:])
X_decoded, z_mu, _ = dyn_learner.encode_decode(X_data[i,:].unsqueeze(0))
print(z_mu)
X_decoded = X_decoded.squeeze()
pybullet_data_generation.show_sample(X_decoded)

In [ ]:
# see how long verification takes
import time
start = time.time()
print(dyn_learner.lyapunov_loss(optimal=opt.lyapunov_loss_optimal))
end = time.time()
print(end - start)

In [ ]:
# train the encoder-dynamics-decoder
import time
start = time.time()
dyn_learner.train(1000, validate=True, device=device, save_rate=5, save_path=os.path.join(folder, variant))
end = time.time()
print(end - start)

In [ ]:
torch.save(dyn_learner.encoder, os.path.join(folder, variant, "encoder"))
torch.save(dyn_learner.decoder, os.path.join(folder, variant, "decoder"))
torch.save(dyn_learner.relu_system.dynamics_relu, os.path.join(folder, variant, "dynamics"))
torch.save(dyn_learner.lyapunov.lyapunov_relu, os.path.join(folder, variant, "lyapunov"))

In [ ]:
dyn_learner.relu_system.dynamics_relu = torch.load(os.path.join(folder, variant, "dynamics")).to('cpu')
dyn_learner.lyapunov.lyapunov_relu = torch.load(os.path.join(folder, variant, "lyapunov")).to('cpu')
dyn_learner.encoder = torch.load(os.path.join(folder, variant, "encoder")).to('cpu')
dyn_learner.decoder = torch.load(os.path.join(folder, variant, "decoder")).to('cpu')

## Some quick checks

In [ ]:
# plot a random rollout
i = np.random.choice(X_data.shape[0], 1)[0]
x_traj, V_traj, z_traj = dyn_learner.rollout(X_data[i, :], opt.validation_rollouts_N, decode_intermediate=False)
for n in range(x_traj.shape[0]):
    pybullet_data_generation.show_sample(x_traj[n, :])

print(V_traj)
plt.plot(V_traj)
plt.show()

print(z_traj)
plt.plot(z_traj[:,0], z_traj[:,1])
plt.show()

In [ ]:
# plot the same rollout but from the real system
x_traj, _ = pbsg.generate_rollout(x_data[i, :], opt.dataset_dt, opt.validation_rollouts_N)
for n in range(x_traj.shape[0]):
    pybullet_data_generation.show_sample(x_traj[n, :])

In [ ]:
# validation over rollouts
plt.plot(dyn_learner.rollout_validation(X_rollouts))
plt.show()

In [ ]:
# computing adversarial samples
z_adv_pos, z_adv_der = dyn_learner.adversarial_samples(optimal=opt.lyapunov_loss_optimal)

In [ ]:
# rolling out the adversarial samples

# z_adv = z_adv_pos
z_adv = z_adv_der
i = np.random.choice(z_adv.shape[0], 1)[0]
X_adv = dyn_learner.decoder(z_adv[i, :].unsqueeze(0))[0, :]
pybullet_data_generation.show_sample(X_adv)

print("*****")

x_traj = dyn_learner.rollout_latent(X_adv, opt.validation_rollouts_N * 2)
for n in range(x_traj.shape[0]):
    pybullet_data_generation.show_sample(x_traj[n, :])

In [ ]:
# plotting the Lyapunov
import neural_network_lyapunov.test.train_2d_lyapunov_utils as train_2d_lyapunov_utils
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(16,16), dpi=100)
ax = fig.add_subplot(111, projection='3d')
train_2d_lyapunov_utils.plot_lyapunov(ax, dyn_learner.lyapunov.lyapunov_relu, opt.V_lambda, opt.z_equilibrium, opt.z_lo, opt.z_up, [20, 20], 1)